In [89]:
from collections import Counter

In [2]:
!head -10 dna.example.fasta

>gi|142022655|gb|EQ086233.1|43 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence
TCGGGCGAAGGCGGCAGCAAGTCGTCCACGCGCAGCGCGGCACCGCGGGCCTCTGCCGTGCGCTGCTTGG
CCATGGCCTCCAGCGCACCGATCGGATCAAAGCCGCTGAAGCCTTCGCGCATCAGGCGGCCATAGTTGGC
GCCAGTGACCGTACCAACCGCCTTGATGCGGCGCTCGGTCATCGCTGCATTGATCGAGTAGCCACCGCCG
CCGCAAATGCCCAGCACGCCAATGCGTTCTTCATCCACATAGGGGAGCGTTACGAGGTAGTCGCAGACCA
CGCGGAAATCCTCGACGCGCAGTGTCGGGTCTTCGGTAAAACGTGGTTCGCCGCCGCTGGCACCCTGGAA
GCTGGCGTCGAAGGCGATGACGACGAAACCTTCCTTGGCCAGCGCCTCGCCATACACGTTCCCCGATGTT
TGCTCCTTGCAGCTGCCGATCGGATGCGCGCTGATGATGGCGGGATATTTCTTGCCTTCGTCGAAGTTCG
GCGGGAAGTGGATGTCGGCTGCGATATCCCAATACACATTCTTGATCTTGACGCTTTTCATGACAGCTCC
GTTCAGGGGGAGGGGGTAAGTTCGCCAGGCCGAATCGTTGGTAGCCAAGCGGCAACGACTCGAATATAGA


1) How many records in the file?

In [1]:
FILE = './dna.example.fasta'

In [2]:
class Record:
    def __init__(self, record_str):
        self.record_str = record_str
        self.header, seq = record_str.split('\n', 1)
        self.seq = seq.replace('\n', '')
        self.a, self.id, self.c, self.call, self.descr = (
            self.header.split('|'))

    def __str__(self):
        return self.record_str
    
    def __len__(self):
        return len(self.seq)

In [87]:
class DNASeq:
    def __init__(self, seq: str):
        self.seq = seq
        self.start_codon = 'ATG'
        self.stop_codons = ['TAA', 'TAG', 'TGA']
        self.n = len(seq)
        
    def to_codons(self, frame_start=1):
        codons = [
            self.seq[i:i+3] for i in range(frame_start - 1, self.n, 3)]
        return [c for c in codons if len(c) == 3]

    def get_orfs(self, frame_start=1):
        'ORF: open reading frame'
        orfs = []
        codons = self.to_codons(frame_start)
        while codons:
            if self.start_codon in codons:
                start = codons.index(self.start_codon)
                first_stop = len(codons) + 1
                for stop in self.stop_codons:
                    if stop in codons[start + 1:]:
                        stop_idx = codons.index(stop, start + 1)
                        if stop_idx < first_stop:
                            first_stop = stop_idx
                if first_stop <= len(codons):
                    orf = ''.join(codons[start:first_stop + 1])
                    orfs.append(orf)
                    codons = codons[first_stop + 1:]
                else:
                    break
            else:
                break
        return orfs
    
    def get_repeats(self, n):
        'n is the length of the repeat sequence'
        repeats = []
        #for start in range(len(self.seq)):
        #    seq = self.seq[start:start + n]
        #    rep = seq
        #    for next_frame_start in range(start + n, len(self.seq), n):
        #        if (self.seq[next_frame_start:next_frame_start + n] 
        #            == seq):
        #            rep += seq
        #        else:
        #            break
        #    if len(rep) > n:
        #        repeats.append(rep)
        # with overlaps:
        for start in range(len(self.seq)):
            seq = self.seq[start:start + n]
            rep = seq
            count = 1
            for next_frame_start in range(start + 1, start + 1 + n):
                if (self.seq[
                        next_frame_start:next_frame_start + n] == seq):
                    rep = self.seq[start:next_frame_start + n]
                    next_frame_start += 1
                    count += 1
                    continue
            if len(rep) > n:
                repeats.append((rep, seq, count))
        return repeats

In [83]:
7/8

0.875

In [84]:
seq = (
    'TCGGGCGAAGGCGGCAGCAAGTCGTCCACGCGCAGCGCGGCACCGCGGGCCTCTGCCGTGCGCTG'
    'CTTGGCCATGGCCTCCAGCGCACCGATCGGATCAAAGCCGCTGAAGCCTTCGCGCATCAGGCGGC'
    'CATAGTTGGCGCCAGTGACCGTACCAACCGCCTTGATGCGGCGCTCGGTCATCGCTGCATTGATC'
    'GAGTAGCCACCGCCGCCGCAAATGCCCAGCACGCCAATGCGTTCTTCATCCACATAGGGGAGCGT'
    'TACGAGGTAGTCGCAGACCACGCGGAAATCCTCGACGCGCAGTGTCGGGTCTTCGGTAAAACGTG'
    'GTTCGCCGCCGCTGGCACCCTGGAAGCTGGCGTCGAAGGCGATGACGACGAAACCTTCCTTGGCC'
    'AGCGCCTCGCCATACACGTTCCCCGATGTTTGCTCCTTGCAGCTGCCGATCGGATGCGCGCTGAT'
    'GATGGCGGGATATTTCTTGCCTTCGTCGAAGTTCGGCGGGAAGTGGATGTCGGCTGCGATATCCC'
    'AATACACATTCTTGATCTTGACGCTTTTCATGACAGCTCCGTTCAGGGGGAGGGGGTAAGTTCGC'
    'CAGGCCGAATCGTTGGTAGCCAAGCGGCAACGACTCGAATATAGA')
seq = DNASeq(seq)
print(seq.get_repeats(3))
print()
seq = DNASeq('ACACA')
print(seq.get_repeats(3))

[['GGCGGC', 'GGC', 2], ['GCAGCA', 'GCA', 2], ['GTCGTC', 'GTC', 2], ['CGCGC', 'CGC', 2], ['GCGCG', 'GCG', 2], ['GCTGCT', 'GCT', 2], ['CGCGC', 'CGC', 2], ['GGCGGC', 'GGC', 2], ['GCGGCG', 'GCG', 2], ['CCGCCG', 'CCG', 2], ['CGCCGC', 'CGC', 2], ['GCCGCC', 'GCC', 2], ['CCGCCG', 'CCG', 2], ['CGCCGC', 'CGC', 2], ['TTCTTC', 'TTC', 2], ['GGGG', 'GGG', 2], ['CGCGC', 'CGC', 2], ['AAAA', 'AAA', 2], ['CGCCGC', 'CGC', 2], ['GCCGCC', 'GCC', 2], ['CCGCCG', 'CCG', 2], ['CGCCGC', 'CGC', 2], ['GACGAC', 'GAC', 2], ['ACGACG', 'ACG', 2], ['CGACGA', 'CGA', 2], ['CCCC', 'CCC', 2], ['GCGCG', 'GCG', 2], ['CGCGC', 'CGC', 2], ['TGATGA', 'TGA', 2], ['GATGAT', 'GAT', 2], ['ATGATG', 'ATG', 2], ['TCGTCG', 'TCG', 2], ['CGGCGG', 'CGG', 2], ['ACACA', 'ACA', 2], ['TTTT', 'TTT', 2], ['GGGGG', 'GGG', 3], ['GGGG', 'GGG', 2], ['GGGGG', 'GGG', 3], ['GGGG', 'GGG', 2], ['ATATA', 'ATA', 2]]

[['ACACA', 'ACA', 2]]


In [5]:
seq.get_orfs(2)

['ATGTTTGCTCCTTGCAGCTGCCGATCGGATGCGCGCTGA']

In [6]:
seq.get_orfs(3)

['ATGCGCGCTGATGATGGCGGGATATTTCTTGCCTTCGTCGAAGTTCGGCGGGAAGTGGATGTCGGCTGCGATATCCCAATACACATTCTTGATCTTGACGCTTTTCATGACAGCTCCGTTCAGGGGGAGGGGGTAAGTTCGCCAGGCCGAATCGTTGGTAGCCAAGCGGCAACGACTCGAATATAG']

In [8]:
def get_records(path):
    with open(path, 'r') as f:
        t = ''.join(f.readlines())
    records = t.split('>')[1:]
    return [Record(record) for record in records]

In [9]:
records = get_records(FILE)
print(len(records))
print()
print(records[-1])

25

gi|142022655|gb|EQ086233.1|101 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence
CACATCGACACGAAGATCACCGCGCATGCGTTGCTGATCACGCTCGTCAGCGCGCGTGCCTCGGACATGA
AGCGATCGATGCCGACCAGCAGTGCGACACCGGCGACGGGCAGGTCGGGCATGACGACGAGCGTGGCGAC
CAGCGCAACCAGCCCGCTTCCGGAAACGCCGGCCGCGCCCTTGGACGTGAGCAGCATGATGGCGAGCATC
ACGGCGATCTGCGACGCGGAAAGGGGCACGTCGCACGCCTGCGCGATGAACAACGCGGCGAGCGTCAGAT
AGATCGCGGTACCGTCCAGATTGAACGAATAACCCGCCGGCAGCACGAGCCCCACGACGCCCTTGTCGCA
CCCGAGCGATTCCAGCTTGACGATCAGGCGTGGCAGAACGGGCTCCGAAGAGGACGTCGCGAGGACGATG
AGCAACTCTTCGCGCAGGTAGCGCAAGAGCCGCCACAGCGCGAAGCCGTGCAGCCGCGCGAGCGGGGCGA
GCACCAGTGCGACGAACAGCCCGCAGGCCACGTAGAAGGACAGCATCAGCTTCGCGAGCGAGCCGATCGA
GCCGATTCCGAAGCGGCCCACCGTGAAGGCCATCGCGCCGAATGCGCCGAGCGGCGCGAGCCGCATGATC
ATCGCGAGCACGCGAAAGACGACCTGGGCGACGCCGTCGATCAGTGCAAGAACGGGCCGCCCGGCCCGCG
GGTGTGCGTTCAGCGAGAAGCCGAACAACAGCGACAGCAGCAGCACCGGCAACACCTCGCCTTTCTCGAA
CGCGCCGAGCATCGTATCGGGGATCACGCTCAGCCCGAACGCGACGAGCCCGTTCGGTTGCGCGTCCCTC
ACGTACGGCGCGAGGATGCGCGA

In [11]:
print(records[0])

gi|142022655|gb|EQ086233.1|43 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence
TCGGGCGAAGGCGGCAGCAAGTCGTCCACGCGCAGCGCGGCACCGCGGGCCTCTGCCGTGCGCTGCTTGG
CCATGGCCTCCAGCGCACCGATCGGATCAAAGCCGCTGAAGCCTTCGCGCATCAGGCGGCCATAGTTGGC
GCCAGTGACCGTACCAACCGCCTTGATGCGGCGCTCGGTCATCGCTGCATTGATCGAGTAGCCACCGCCG
CCGCAAATGCCCAGCACGCCAATGCGTTCTTCATCCACATAGGGGAGCGTTACGAGGTAGTCGCAGACCA
CGCGGAAATCCTCGACGCGCAGTGTCGGGTCTTCGGTAAAACGTGGTTCGCCGCCGCTGGCACCCTGGAA
GCTGGCGTCGAAGGCGATGACGACGAAACCTTCCTTGGCCAGCGCCTCGCCATACACGTTCCCCGATGTT
TGCTCCTTGCAGCTGCCGATCGGATGCGCGCTGATGATGGCGGGATATTTCTTGCCTTCGTCGAAGTTCG
GCGGGAAGTGGATGTCGGCTGCGATATCCCAATACACATTCTTGATCTTGACGCTTTTCATGACAGCTCC
GTTCAGGGGGAGGGGGTAAGTTCGCCAGGCCGAATCGTTGGTAGCCAAGCGGCAACGACTCGAATATAGA
GAGCCGATTGGAATTCCGTAAGATCGCAATCTGGACTACAGTGGTATCTTCAAATTGACAATGGCACCTA
CATGGATCCCTCACTGCTTCCGTCTCTCGCGTGGTTCGCCCACGTCGCACATCATCGTAGCTTCACGAAA
GCGGCTGCGGAAATGGGCGTTTCTCGAGCAAACCTGTCGCAGAACGTGAAGGCGCTCGAACGCCGGTTGA
ACGTCAAGCTGCTGTATCGAACGACTCG

2) What are the lengths of the records?  Shortest?  Longest?

In [12]:
for r in records:
    print(len(r))

990
724
3080
2863
3832
4805
1663
512
691
3072
1801
3603
2478
1608
4745
1810
3424
1451
3276
2124
1712
1325
1189
555
2449


In [13]:
def get_longest(records):
    longest = 0
    out = []
    for record in records:
        seq_len = len(record)
        if seq_len > longest:
            longest = seq_len
            out = [record]
        elif seq_len == longest:
            out.append(record)
    return out

In [14]:
def get_shortest(records):
    shortest = 9e999
    out = []
    for record in records:
        seq_len = len(record)
        if seq_len < shortest:
            shortest = seq_len
            out = [record]
        elif seq_len == shortest:
            out.append(record)
    return out

In [15]:
longest = get_longest(records)
shortest = get_shortest(records)

print('longest:')
for r in longest:
    print(r.header, len(r))
print('\nshortest:')
for r in shortest:
    print(r.header, len(r))

longest:
gi|142022655|gb|EQ086233.1|323 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence 4805

shortest:
gi|142022655|gb|EQ086233.1|521 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence 512


3) Get longest ORF

In [30]:
def get_longest_orf(records, frame_start):
    longest = 0
    header = None
    for record in records:
        seq = DNASeq(record.seq)
        orfs = seq.get_orfs(frame_start)
        if orfs:
            n = max([len(o) for o in orfs])
            if n > longest:
                longest = n
                header = record.header
    return longest, header

In [33]:
longest, header = get_longest_orf(records, 1)
print(f'longest: {longest}\n{header}')

longest: 1686
gi|142022655|gb|EQ086233.1|323 marine metagenome JCVI_SCAF_1096627390048 genomic scaffold, whole genome shotgun sequence


4) Get repeats

In [98]:
counts = {}

for record in records:
    seq = DNASeq(record.seq)
    repeats = seq.get_repeats(3)
    for r in repeats:
        pattern = r[1]
        if not isinstance(pattern, str):
            print('\n\nERROR:')
            print(pattern)
            print(record.seq)
            break
        reps = r[2]
        key = (pattern, reps)
        if key in counts:
            counts[key] = counts[key] + 1
        else:
            counts[key] = 1
    #print(repeats)
    #print()

In [100]:
most_common = max(counts.values())
for k, v in counts.items():
    if v == most_common:
        print(k)

('GCG', 2)
